In [ ]:
include("_.jl")

In [ ]:
an_hu_me = TableAccess.read(joinpath(PAO, "human_gene_by_method.tsv"))

hu_ = an_hu_me[!, "Gene"]

length(hu_)

In [ ]:
na = "Human Immune Population"

se_im_ = sort(
    convert(
        Dict{String,Vector{String}},
        DictExtension.read(joinpath(PAI, "set_to_genes.json")),
    ),
)

im_ = vcat(values(se_im_)...)

length(im_)

In [ ]:
co_ = intersect(im_, hu_)

In [ ]:
se_ = [join([se for (se, im_) in se_im_ if co in im_], "|") for co in co_]

In [ ]:
hg_gr = Gene.map_to_column(coalesce.(Gene.read_hgnc(), "?"), ["gene_group", "symbol"])

gr_ = Vector{String}()

for co in co_

    if haskey(hg_gr, co)

        gr = hg_gr[co]

    else

        println(co)

        gr = "??"

    end

    push!(gr_, gr)

end

gr_

In [ ]:
an_ge_ir = DataFrame("Gene" => co_, na => se_, "Group" => gr_)

TableAccess.write(joinpath(PAO, "gene_by_information.tsv"), an_ge_ir)

an_ge_ir

## Plot

In [ ]:
he = 1390

wi = round(Int64, he * MathConstants.golden)

axis_title_font_size = 24

la = Layout(
    barmode = "stack",
    height = he,
    width = wi,
    margin_b = 160,
    title = attr(text = string("Number of Human Genes in ", na, "s"), font_size = 32),
    yaxis_title = attr(text = "Number of Human Genes", font_size = axis_title_font_size),
    xaxis = attr(title = attr(text = na, font_size = axis_title_font_size), tickangle = 60),
)

nac = PathExtension.clean(na)

;

In [ ]:
se_n = sort(countmap(an_ge_ir[!, na]); byvalue = true)

In [ ]:
x = collect(keys(se_n))

y = collect(Float64, values(se_n))

pl = FinalFigure.plot_bar([y], [x], la = la)

FinalFigure.write(joinpath(PAO, string("human_", nac, "_genes.png")), pl, he, wi, 8)

In [ ]:
sort!(an_ge_ir, order(na; by = po -> se_n[po]))

name_ = Vector{String}()

y_ = Vector{Vector{Float64}}()

for gr in unique([split(gr, "|")[1] for gr in an_ge_ir[!, "Group"] if !(gr in ["?", "??"])])

    push!(name_, gr)

    push!(
        y_,
        [
            sum(getindex.(split.(df[!, "Group"], "|"), 1) .== gr) for
            df in groupby(an_ge_ir, na)
        ],
    )

end

pl = FinalFigure.plot_bar(y_, [x for it = 1:length(y_)]; name_ = name_, la = la)

FinalFigure.write(
    joinpath(PAO, string("human_", nac, "_genes_by_group.png")),
    pl,
    he,
    wi,
    8,
)